In [40]:
import numpy as np
import os
import pandas as pd
import sklearn

from sklearn import preprocessing
data1 = pd.read_csv('health_data.csv')
data1.head()

,Age,Sex,HighChol,CholCheck,BMI,Smoker,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,GenHlth,MentHlth,PhysHlth,DiffWalk,Diabetes,Hypertension,Stroke
0,4.0,1.0,0.0,1.0,26.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,5.0,30.0,0.0,0.0,1.0,0.0
1,12.0,1.0,1.0,1.0,26.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0
2,13.0,1.0,0.0,1.0,26.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0
3,11.0,1.0,1.0,1.0,28.0,1.0,0.0,1.0,1.0,1.0,0.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0
4,8.0,0.0,0.0,1.0,29.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


data1.dtype()

In [41]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   70692 non-null  float64
 1   Sex                   70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   HeartDiseaseorAttack  70692 non-null  float64
 7   PhysActivity          70692 non-null  float64
 8   Fruits                70692 non-null  float64
 9   Veggies               70692 non-null  float64
 10  HvyAlcoholConsump     70692 non-null  float64
 11  GenHlth               70692 non-null  float64
 12  MentHlth              70692 non-null  float64
 13  PhysHlth              70692 non-null  float64
 14  DiffWalk              70692 non-null  float64
 15  Diabetes           

In [42]:
data1.shape

(70692, 18)

In [43]:
label_encoder = preprocessing.LabelEncoder()
data1["Age"]= label_encoder.fit_transform(data1["Age"])
data1["Age"].unique()

array([ 3, 11, 12, 10,  7,  0,  5,  2,  6,  9,  8,  4,  1])

In [44]:
data1["Age"].value_counts()

9     10856
8     10112
7      8603
10     8044
6      6872
12     5426
11     5394
5      4648
4      3520
3      2793
2      2049
1      1396
0       979
Name: Age, dtype: int64

In [59]:
# Stratified Sampling using Scikit-learn's Stratified Shuffle Split Class
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in split.split(data1, data1["Stroke"]):
    strat_train_set1 = data1.loc[train_index]
    strat_test_set1 = data1.loc[test_index]

In [60]:
train_set1 = strat_train_set1.drop("Stroke", axis=1) 
train_set1 = strat_train_set1.drop("Age", axis=1)  # drop labels for training set
train_labels = strat_train_set1["Stroke"].copy()
test_set1 = strat_test_set1.drop("Stroke", axis=1) # drop labels for testing set
test_set1 = strat_test_set1.drop("Age", axis=1)
test_labels = strat_test_set1["Stroke"].copy()

In [61]:
from sklearn.decomposition import PCA
pca = PCA(n_components=17, svd_solver="full")
train_set = pca.fit_transform(train_set1)
test_set = pca.transform(test_set1)

In [62]:
test_set1.shape

(17673, 17)

In [63]:
train_set1.shape

(53019, 17)

In [64]:
from sklearn.metrics import matthews_corrcoef
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
#parameters = {'reg_paramfloat':[0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0], 'store_covariance':['True', 'False']}
clf = QuadraticDiscriminantAnalysis()
res1=clf.fit(train_set1, train_labels)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [65]:
clf.predict(train_set1)
clf.get_params()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:912: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:912: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:915: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])


{'priors': None, 'reg_param': 0.0, 'store_covariance': False, 'tol': 0.0001}

In [66]:
clf.score(train_set, train_labels)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but QuadraticDiscriminantAnalysis was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:912: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:912: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:915: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])


0.9378336068201966

In [67]:
y_test_pred=clf.predict(test_set)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but QuadraticDiscriminantAnalysis was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:912: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:912: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:915: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])


In [68]:
matthews_corrcoef(test_labels, y_test_pred)

0.0

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, y_test_pred)

0.9378147456572172

In [70]:
from sklearn.metrics import precision_score
precision_score(test_labels, y_test_pred, average=None)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([0.93781475, 0.        ])

In [71]:
from sklearn.metrics import f1_score
f1_score(test_labels, y_test_pred, average=None)

array([0.9679096, 0.       ])

In [58]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(test_labels,y_test_pred)

0.5